In [1]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import drive
drive.mount("drive")

Dataset_PATH = "drive/My Drive/RPS/image_data"

Mounted at drive


In [1]:
import os
from tqdm.notebook import tqdm
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from keras.utils.np_utils import to_categorical
from tensorflow.keras.layers import Activation, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
Processed_PATH = "drive/My Drive/RPS-processed"
#os.mkdir(Processed_PATH)

In [5]:
labels = ["rock","paper","scissor","none"]

for label in labels:
  path = os.path.join(Dataset_PATH, label)
  for flie_name in tqdm(os.listdir(path)):
    try:
      img = cv2.imread(os.path.join(path, flie_name))
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      img = cv2.resize(img, (224, 224))
      cv2.imwrite(os.path.join(Processed_PATH,label)+"/"+flie_name,img)
    except:
      continue

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

In [5]:
Data_Generator = ImageDataGenerator(horizontal_flip=True,
                                    brightness_range=[0.5,1.25],
                                    zoom_range=[0.8,1],
                                    rotation_range = 15,
                                    validation_split=.2)

In [6]:
Train_Data_Generator = Data_Generator.flow_from_directory(Processed_PATH,
                                                          target_size=(224,224),
                                                          class_mode="categorical",
                                                          shuffle=True,
                                                          batch_size=32)

Found 280 images belonging to 4 classes.


In [7]:
base_network  = ResNet50(input_shape = (224,224,3),weights='imagenet',include_top=False)
base_network.trainable = False

94781440/94765736 [==============================] - 0s 0us/step


In [15]:
def get_model():
    model = Sequential([
        base_network,
        tf.keras.layers.AveragePooling2D(pool_size = (7,7)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation = 'relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(4, activation = 'softmax')
    ])
    return model


# define the model
model = get_model()
model.compile(
    optimizer=Adam(lr=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [16]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 average_pooling2d_2 (Averag  (None, 1, 1, 2048)       0         
 ePooling2D)                                                     
                                                                 
 flatten_2 (Flatten)         (None, 2048)              0         
                                                                 
 dense_4 (Dense)             (None, 128)               262272    
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 4)                 516       
                                                      

In [17]:
epochs_ = 13

# start training
model.fit(Train_Data_Generator,
          steps_per_epoch=(Train_Data_Generator.n//32),
          epochs = epochs_)

Epoch 1/13
8/8 [==============================] - 7s 478ms/step - loss: 1.8781 - accuracy: 0.2944
Epoch 2/13
8/8 [==============================] - 4s 454ms/step - loss: 1.1793 - accuracy: 0.4637
Epoch 3/13
8/8 [==============================] - 4s 445ms/step - loss: 0.6611 - accuracy: 0.7823
Epoch 4/13
8/8 [==============================] - 4s 445ms/step - loss: 0.4397 - accuracy: 0.8790
Epoch 5/13
8/8 [==============================] - 4s 448ms/step - loss: 0.3320 - accuracy: 0.9234
Epoch 6/13
8/8 [==============================] - 4s 461ms/step - loss: 0.2815 - accuracy: 0.9355
Epoch 7/13
8/8 [==============================] - 4s 451ms/step - loss: 0.2212 - accuracy: 0.9597
Epoch 8/13
8/8 [==============================] - 4s 455ms/step - loss: 0.1854 - accuracy: 0.9718
Epoch 9/13
8/8 [==============================] - 4s 451ms/step - loss: 0.1296 - accuracy: 0.9798
Epoch 10/13
8/8 [==============================] - 4s 468ms/step - loss: 0.1345 - accuracy: 0.9844
Epoch 11/13
8/8 [==

In [18]:
# save the model for later use
model.save("drive/My Drive/RPS/RPS-model.h5")

In [5]:
from keras.models import load_model
path = 'drive/My Drive/RPS/RPS-model.h5'
model = load_model(path)

In [17]:
test_img = '/content/2.jpg'

In [28]:
map = {
    0 : 'none',
    1 : 'paper',
    2 : 'rock',
    3 : 'scissor'
}

def mapper(val):
  return map[val]

In [29]:
img = cv2.imread(test_img)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (224, 224))

# predict the move made
pred = model.predict(np.array([img]))
move_code = np.argmax(pred[0])
move_name = mapper(move_code)

print("Predicted: {}".format(move_name))

Predicted: none
